In [1]:
import re
import pandas as pd
import numpy as np
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
import gensim
from gensim import corpora, models
from timeit import default_timer as timer
from datetime import datetime, timedelta
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import spacy
from collections import Counter
import pyLDAvis
import pyLDAvis.gensim
from sklearn.model_selection import train_test_split

importlib.reload(funcs)

<module 'funcs' from 'D:\\_projekty\\praca\\funcs.py'>

In [2]:
dane = pd.read_csv('00_dane/dane_TM.csv',  sep='|', encoding='UTF-8')
dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: re.sub(r'[\W\d]',' ', x.lower()))
nlp = spacy.load('pl_model')
stop = set(nlp.Defaults.stop_words)
stop.add('zł')
stop.add('pkt')
stop.add('art')
stop.add('ustawy')
stop.add('r')
dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: [word for word in x.split() if word not in stop])

Ponieważ pusta wartość jest interpretowana jako float, należy ograniczyć zbiór.

In [42]:
dane = dane[dane['full_name'].apply(lambda x:isinstance(x, str))]

In [3]:
dane.head(3)

,nr,date_,speech_rawer,full_name,party_final,speech_rawest,topic_LDA_BOW,topic_LDA_TF,opis_LDA_BOW,opis_LDA_TF
0,0,01-02-1992,Chciałbym kilka zdań powiedzieć na temat skut...,Aleksander Bentkowski,Polskie Stronnictwo Ludowe,"[chcieć, zdanie, powiedzieć, temat, skutek, us...",16.0,13.0,wymiar sprawiedliwości,przepisy
1,1,01-02-1992,"Przysłuchując się tej bardzo emocjonalnej, dł...",Aleksander Kwaśniewski,Sojusz Lewicy Demokratycznej,"[przysłuchiwać, emocjonalny, długo, trwać, deb...",14.0,16.0,ustrój państwa,ustrój państwa
2,2,01-02-1992,"Różne przebyliśmy drogi, żeby tu dzisiaj powi...",Alojzy Pietrzyk,Klub Parlamentarny NSZZ Solidarność,"[różny, przebyć, droga, powiedzieć, swój, ocze...",14.0,0.0,ustrój państwa,polityka


Utworzenie słownika autor - dokument

In [45]:
aut2doc=dict()
for (aut, doc_nr) in zip(dane['full_name'], dane['nr']):
    if aut not in aut2doc.keys():
        aut2doc[aut]=[]
    aut2doc[aut].append(doc_nr)

In [5]:
aut2doc['Janusz Korwin-Mikke']

[996,
 997,
 998,
 1379,
 1380,
 2154,
 2155,
 4046,
 4047,
 4656,
 4657,
 4658,
 4659,
 5133,
 6148,
 6149,
 6620,
 6621,
 7339,
 7390,
 8486,
 8487,
 8662,
 8663,
 10056,
 10057,
 10280,
 10557,
 11844,
 11845,
 12935,
 12936,
 14928,
 14929,
 15381,
 15382,
 15453,
 15454,
 18338,
 18996,
 20692,
 20693,
 21308,
 21367,
 21995,
 21996,
 22389,
 22390,
 24580,
 24581,
 24582,
 24583,
 26094,
 27458,
 27459,
 29797,
 29798,
 29799,
 30154,
 31280,
 31281,
 31282,
 31283,
 32914,
 32915,
 35708,
 35709,
 35710,
 37321,
 39230,
 39231,
 39232,
 39233,
 43065,
 46908,
 49073,
 49957,
 49958,
 53852,
 53853,
 55814,
 60277,
 61560,
 65579,
 67265,
 72926,
 72927,
 73197,
 73198,
 73199,
 74338,
 74339,
 77502,
 77503,
 78409,
 78410,
 78411,
 79286,
 79287,
 79288,
 79289,
 81882,
 81883,
 81884,
 82941,
 83147,
 83148,
 83914,
 83915,
 83979,
 83980,
 83981,
 85054,
 85791,
 85792,
 88129,
 89176,
 89177,
 89252,
 89253,
 90241,
 90242,
 90243,
 91093,
 91094,
 91171,
 91172,
 92441,
 92

In [6]:
doc2aut = dane['full_name'].T.to_dict()

In [7]:
doc2aut[122846]

'Janusz Korwin-Mikke'

## Przygotowanie słowników itp.

In [70]:
dane = dane.reset_index()

In [71]:
dane.drop('index', axis=1)

,nr,date_,speech_rawer,full_name,party_final,speech_rawest,topic_LDA_BOW,topic_LDA_TF,opis_LDA_BOW,opis_LDA_TF
0,0,01-02-1992,Chciałbym kilka zdań powiedzieć na temat skut...,Aleksander Bentkowski,Polskie Stronnictwo Ludowe,"[chcieć, zdanie, powiedzieć, temat, skutek, us...",16.0,13.0,wymiar sprawiedliwości,przepisy
1,1,01-02-1992,"Przysłuchując się tej bardzo emocjonalnej, dł...",Aleksander Kwaśniewski,Sojusz Lewicy Demokratycznej,"[przysłuchiwać, emocjonalny, długo, trwać, deb...",14.0,16.0,ustrój państwa,ustrój państwa
2,2,01-02-1992,"Różne przebyliśmy drogi, żeby tu dzisiaj powi...",Alojzy Pietrzyk,Klub Parlamentarny NSZZ Solidarność,"[różny, przebyć, droga, powiedzieć, swój, ocze...",14.0,0.0,ustrój państwa,polityka
3,3,01-02-1992,"Wszystkim nam bardzo zależy na tym, aby urato...",Alojzy Szablewski,Klub Parlamentarny Zjednoczenia Chrześcijańsko...,"[wszystek, zależeć, uratować, przemysł, stocze...",10.0,5.0,NaN,przedsiębiorstwa
4,4,01-02-1992,Z brzmienia pkt. 1 projektu uchwały odczytuję...,Andrzej Baraniecki,Sojusz Lewicy Demokratycznej,"[brzmię, punkt, projekt, uchwała, odczytować, ...",10.0,16.0,NaN,ustrój państwa
...,...,...,...,...,...,...,...,...,...,...
158315,158874,31-08-2011,Sojuszowi Lewicy Demokratycznej bardzo zależy...,Romuald Ajchler,Sojusz Lewicy Demokratycznej,"[sojusz, lewica, demokratyczny, zależeć, ziemi...",26.0,8.0,posiedzenie,posiedzenie
158316,158875,31-08-2011,Panie Przewodniczący Krajowej Rady Sądownictw...,Stanisław Rydzoń,Sojusz Lewicy Demokratycznej,"[przewodniczący, krajowy, rada, sądownictwo, z...",16.0,13.0,wymiar sprawiedliwości,przepisy
158317,158876,31-08-2011,Mam pytanie do pana ministra. Panie ministrze...,Stanisław Rydzoń,Sojusz Lewicy Demokratycznej,"[mieć, pytanie, minister, minister, mieć, doty...",28.0,2.0,handel,inwestycje
158318,158877,31-08-2011,Pani Minister! Bardzo dużo mówi się o szkolni...,Stanisław Stec,Sojusz Lewicy Demokratycznej,"[minister, mówić, szkolnictwo, zawodowy, dosto...",7.0,2.0,NaN,inwestycje


In [72]:
aut2doc=dict()
for (aut, doc_nr) in zip(dane['full_name'], dane.index):
    if aut not in aut2doc.keys():
        aut2doc[aut]=[]
    aut2doc[aut].append(doc_nr)

In [54]:
aut2doc=dict()
for (aut, doc_nr) in zip(dane['full_name'], dane.index):
    if aut not in aut2doc.keys():
        aut2doc[aut]=[]
    aut2doc[aut].append(doc_nr)
doc2aut = dane['full_name'].T.to_dict()
corp = dane['speech_rawest']
bigram = gensim.models.Phrases(corp, min_count=20, threshold=50)
bigram_mod = gensim.models.phrases.Phraser(bigram)
corp = [bigram_mod[doc] for doc in corp]
dictionary = gensim.corpora.Dictionary(corp)
dictionary.filter_extremes(no_above=0.2, no_below=50)
dictionary.compactify()
pickle.dump(dictionary, open('dict_atm.p', 'wb'))
BOW = [dictionary.doc2bow(text) for text in corp]
pickle.dump(BOW, open('bow_atm.p', 'wb'))

In [60]:
dictionary[:100]

TypeError: unhashable type: 'slice'

In [25]:
from gensim.corpora import mmcorpus
from gensim.test.utils import common_dictionary, datapath, temporary_file
print(mmcorpus.MmCorpus(datapath('testcorpus.mm'))[8])

[(4, 1.0), (10, 1.0), (11, 1.0)]


In [73]:
from gensim.models import AuthorTopicModel
%time model = AuthorTopicModel(corpus=BOW, num_topics=40,  \
                author2doc=aut2doc, passes=5, eval_every=0, \
                iterations=1, random_state=1)

2020-06-29 21:25:33 WARNING: no word id mapping provided; initializing from corpus, assuming identity


Wall time: 10min 24s


In [74]:
pickle.dump(model, open('model_atm.p', 'wb'))

In [75]:
author_vecs = [model.get_author_topics(author) for author in model.id2author.values()]

In [79]:
model.get_author_topics('Janusz Korwin-Mikke')

[(3, 0.020907169204631332),
 (4, 0.015689194848021278),
 (7, 0.11814505194455387),
 (8, 0.12721065281124205),
 (9, 0.019493240941848858),
 (10, 0.01629923530548447),
 (11, 0.128871483270611),
 (13, 0.20748287154563094),
 (19, 0.02169934248989301),
 (20, 0.01470360538579779),
 (21, 0.04436054715072491),
 (22, 0.011845791090854364),
 (23, 0.08962492538245025),
 (31, 0.03806609079975118),
 (34, 0.019873508555917928),
 (39, 0.02336882080154783)]

In [83]:
for a in model.get_topics():
    print(a)

[3.26072294e-08 4.19542806e-08 3.26072294e-08 ... 3.26072294e-08
 3.26072294e-08 3.26072294e-08]
[1.93734994e-04 3.31498649e-08 7.23282755e-08 ... 3.31498649e-08
 3.31498649e-08 3.31498649e-08]
[4.50689885e-08 1.05594491e-04 4.50689885e-08 ... 4.50689885e-08
 1.02159394e-04 4.50689885e-08]
[4.82672262e-08 6.67869325e-04 4.82672262e-08 ... 9.75816202e-05
 4.82672262e-08 4.82672262e-08]
[2.34001845e-08 2.34001927e-08 2.34001845e-08 ... 2.34001845e-08
 2.34001845e-08 2.34001845e-08]
[3.32370077e-08 3.32370122e-08 1.10221275e-05 ... 3.32370077e-08
 3.32370077e-08 3.32370077e-08]
[3.93590509e-08 3.93590505e-08 2.36230279e-05 ... 3.93590505e-08
 3.93590505e-08 3.93590505e-08]
[7.47360824e-05 5.44484823e-03 2.68867799e-08 ... 2.68867799e-08
 2.68867799e-08 2.68867799e-08]
[1.89674423e-08 4.33109969e-06 1.89674387e-08 ... 1.89674387e-08
 1.89674387e-08 1.89674387e-08]
[4.99211356e-05 3.53852744e-08 3.53852743e-08 ... 3.53852743e-08
 3.53852743e-08 3.53852743e-08]
[3.39579587e-08 3.43282455e-08